<a href="https://colab.research.google.com/github/Amikuto/DAaML/blob/master/06_CNN_embeddings_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn

# 6. Классификация текстов при помощи сверточных сетей

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [121]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [122]:
first_sen = nltk.sent_tokenize(text)[0].replace(".", "").lower()
first_sen

'select your preferences and run the install command'

In [123]:
words = nltk.word_tokenize(first_sen)
words

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

In [124]:
words_dict = {k: v for v, k in enumerate(words)}
words_dict

{'select': 0,
 'your': 1,
 'preferences': 2,
 'and': 3,
 'run': 4,
 'the': 5,
 'install': 6,
 'command': 7}

In [125]:
[words_dict[i] for i in words]

[0, 1, 2, 3, 4, 5, 6, 7]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [126]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [127]:
all_words = nltk.word_tokenize(text.lower().replace(".", ""))
all_words_dict = {k: v for v, k in enumerate(all_words)}
all_words_dict

{'select': 0,
 'your': 1,
 'preferences': 2,
 'and': 14,
 'run': 4,
 'the': 10,
 'install': 6,
 'command': 7,
 'stable': 8,
 'represents': 9,
 'most': 11,
 'currently': 12,
 'tested': 13,
 'supported': 15,
 'version': 16,
 'of': 17,
 'pytorch': 18,
 'note': 19,
 'that': 20,
 'libtorch': 21,
 'is': 22,
 'only': 23,
 'available': 24,
 'for': 25,
 'c++': 26}

In [128]:
first_sen = nltk.sent_tokenize(text)[0].replace(".", "").lower()
first_sen_words = nltk.word_tokenize(first_sen)
first_sen_words

['select', 'your', 'preferences', 'and', 'run', 'the', 'install', 'command']

In [129]:
tensor = torch.zeros(len(first_sen_words), len(all_words_dict))

In [130]:
for i, word in enumerate(first_sen_words):
  tensor[i][all_words_dict[word]] = 1
  print(all_words_dict[word])

0
1
2
14
4
10
6
7


In [131]:
tensor

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])

In [132]:
tensor.shape

torch.Size([8, 25])

1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [140]:
from collections import Counter

vocab = Counter(all_words_dict)
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_size = len(vocab)
vocab_size

25

In [139]:
word2idx = {word: ind for ind, word in enumerate(vocab)}
word2idx

{'c++': 0,
 'for': 1,
 'available': 2,
 'only': 3,
 'is': 4,
 'libtorch': 5,
 'that': 6,
 'note': 7,
 'pytorch': 8,
 'of': 9,
 'version': 10,
 'supported': 11,
 'and': 12,
 'tested': 13,
 'currently': 14,
 'most': 15,
 'the': 16,
 'represents': 17,
 'stable': 18,
 'command': 19,
 'install': 20,
 'run': 21,
 'preferences': 22,
 'your': 23,
 'select': 24}

In [143]:
encoded_sentences = [word2idx[word] for word in first_sen_words]
encoded_sentences

[24, 23, 22, 12, 21, 16, 20, 19]

In [147]:
emb_dim = 25
emb_layer = nn.Embedding(vocab_size, emb_dim)
word_vectors = emb_layer(torch.LongTensor(encoded_sentences))
word_vectors

tensor([[ 0.8165,  0.2662, -1.1366, -0.6337, -1.0894,  0.8706,  0.1666,  0.0857,
          0.8355, -0.3842, -0.2785, -0.2986,  0.8179, -0.0656, -1.2223,  0.7226,
          1.2288,  0.8772,  1.3329,  1.6523,  0.1787, -0.6863, -0.5514, -1.7908,
          0.6648],
        [-1.0429,  1.5159,  0.0171,  0.2410, -0.4140, -1.9278, -0.0953, -0.2972,
         -0.5538, -0.9271, -2.1029,  0.6576, -0.1910, -1.0597,  0.2231, -0.5538,
          0.2335,  0.5641,  0.3492,  0.1145, -1.0163, -0.4078,  0.8625, -0.7429,
          0.7290],
        [ 0.4876,  0.2572, -0.5709,  1.7909,  0.5164, -1.1272, -1.0606,  0.6148,
         -1.0395,  0.4794,  0.6611, -0.3650,  0.0302,  1.2100, -0.3174, -0.0062,
         -0.3299,  2.2702, -0.9089, -0.5905, -0.8196, -0.9299, -0.0580,  0.3652,
          0.5199],
        [ 0.1369, -1.5774, -0.5812, -0.3708, -0.4686, -0.7681,  1.4158, -0.0580,
          0.4623,  0.2257, -0.5860,  0.3584, -0.8206,  0.1281, -0.5581,  2.1217,
         -2.0848,  2.6783, -0.9423,  0.0971, -0.8848

In [148]:
word_vectors.shape

torch.Size([8, 25])

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [176]:
surname_dataset = pd.read_csv("surnames/surnames.csv")
surname_dataset.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


In [177]:
surname_dict = pd.Series(surname_dataset.nationality.unique()).to_dict()
surname_dict = dict(map(reversed, surname_dict.items()))
surname_dict

{'English': 0,
 'French': 1,
 'Arabic': 2,
 'Russian': 3,
 'Japanese': 4,
 'Chinese': 5,
 'Italian': 6,
 'Czech': 7,
 'Irish': 8,
 'German': 9,
 'Greek': 10,
 'Spanish': 11,
 'Polish': 12,
 'Dutch': 13,
 'Vietnamese': 14,
 'Korean': 15,
 'Portuguese': 16,
 'Scottish': 17}

In [178]:
dataset_nation_as_index = surname_dataset.copy()
dataset_nation_as_index.nationality = surname_dataset.nationality.map(lambda x: surname_dict[x])
dataset_nation_as_index

,surname,nationality
0,Woodford,0
1,Coté,1
2,Kore,0
3,Koury,2
4,Lebzak,3
...,...,...
10975,Quraishi,2
10976,Innalls,0
10977,Król,12
10978,Purvis,0


In [179]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset_nation_as_index["surname"], dataset_nation_as_index["nationality"], test_size=0.2)

In [180]:
X_train

3876          Yamagata
10923          Breiner
3663       Likhovskikh
188             Gassiy
5228          Rochford
             ...      
878           Hasegawa
3551             Neish
8284            Abbott
10202           Keelan
7947     Abramtchikoff
Name: surname, Length: 8784, dtype: object

In [181]:
y_train

3876     4
10923    9
3663     3
188      3
5228     0
        ..
878      4
3551     0
8284     0
10202    0
7947     3
Name: nationality, Length: 8784, dtype: int64

In [182]:
class Vocab:
  def __init__(self, column: pd.DataFrame | pd.Series):
    all_chars = pd.Series(column.values).map(lambda x: list(x.lower())).explode().unique()
    all_chars = np.insert(all_chars, 0, "<PAD>")
    self.idx_to_token = {index: token for index, token in enumerate(all_chars)}
    self.token_to_idx = {token: index for index, token in enumerate(all_chars)}
    self.max_seq_len = len(all_chars)

In [183]:
vocab = Vocab(dataset_nation_as_index["surname"])
vocab.max_seq_len

56

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса>
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

In [204]:
from torch.utils.data import Dataset


class SurnamesDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab
    self.max_X = 17
    self.max_y = 10

  def vectorize(self, surename):
    tensor = torch.zeros(vocab.max_seq_len)
    for i, val in enumerate(surename):
      tensor[i] = vocab.token_to_idx[val.lower()]
    tensor[tensor==0] = vocab.token_to_idx["<PAD>"]
    return tensor

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    surname = self.X.iloc[idx]

    nation = self.y.iloc[idx]

    return self.vectorize(surname), nation

In [205]:
dataset = SurnamesDataset(X=X_train, y=y_train, vocab=vocab)

In [207]:
X_train.iloc[0], y_train.iloc[0]

('Yamagata', 4)

In [208]:
dataset[0]

(tensor([12., 16., 20., 16., 24., 16.,  7., 16.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 4)

## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%